In [ ]:
%load_ext autoreload
%autoreload 2

import logging

import helpers.hdbg as hdbg
import helpers.hprint as hprint

hprint.config_notebook()

# hdbg.init_logger(verbosity=logging.DEBUG)
hdbg.init_logger(verbosity=logging.INFO)
# hdbg.test_logger()
_LOG = logging.getLogger(__name__)

# Generate random data

In [ ]:
import numpy as np

import oms.pnl_simulator as opnlsimu

df = opnlsimu.get_random_market_data(21)

display(df.head(3))
display(df.tail(3))

## Lag-based PnL vs Level1 simulation

In [ ]:
mode = "instantaneous"
df_5mins = opnlsimu.resample_data(df, mode)
display(df_5mins)

In [ ]:
df.plot()

In [ ]:
df, df_5mins = opnlsimu.get_example_market_data1()

display(df_5mins)

In [ ]:
# Compute pnl using simulation.
w0 = 100.0
final_w, tot_ret, df_5mins = opnlsimu.compute_pnl_level1(w0, df, df_5mins)

print(final_w, tot_ret)

In [ ]:
# Compute pnl using lags.
# df_5mins["pnl"] = df_5mins["preds"] * df_5mins["ret_0"].shift(-2)
# tot_ret2 = (1 + df_5mins["pnl"]).prod() - 1
# display(df_5mins[:-1])

tot_ret2, df_5mins = opnlsimu.compute_lag_pnl(df_5mins)

# Check that the results are the same.
print("tot_ret=", tot_ret)
print("tot_ret2=", tot_ret2)
np.testing.assert_almost_equal(tot_ret, tot_ret2)

## Lag-based PnL vs Level1 vs Level2 simulation

In [ ]:
mode = "instantaneous"
df, df_5mins = opnlsimu.get_example_market_data1()

# Level 1 sim.
initial_wealth = 1000
final_w, tot_ret, df_5mins = opnlsimu.compute_pnl_level1(
    initial_wealth, df, df_5mins
)
# Lag-based sim.
tot_ret2, df_5mins = opnlsimu.compute_lag_pnl(df_5mins)

# Level 2 sim.
config = {
    "price_column": "price",
    "future_snoop_allocation": True,
    "order_type": "price.end",
}

df_5mins = opnlsimu.compute_pnl_level2(df, df_5mins, initial_wealth, config)

df_5mins